In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import duckdb
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

year = 2025
now = str(datetime.now().date() - timedelta(days=1))
print(f"Today's date:", now)

con = duckdb.connect(database=":memory:")
stats = ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA', 'TPM', 'STL', 'BLK', 'STL_BLK']

Today's date: 2025-12-15


In [2]:
%run ./common_utils.ipynb

In [ ]:
def create_actuals_tbl(con):
    df_lines = pd.read_csv(f"../tables/{year}/parlay_lines.csv")
    df_lines['Date'] = pd.to_datetime(df_lines.Date)
    df_lines = df_lines[(df_lines.Date == now)]
    df_log = pd.read_csv(f"../tables/{year}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df_log['STL_BLK'] = df_log.STL + df_log.BLK
    df = con.execute("""SELECT t2.*, t1.* EXCLUDE(Date, Team, Player, Pos) FROM df_log t1 JOIN df_lines t2 
                        ON t1.Team = t2.Team AND t1.Player = t2.Player""").fetchdf()
    cols = []
    for stat in stats:
        df[f'{stat}_Result'] = np.where(df[stat] > df[f'{stat}_line'], 'O', 'U')
        df[f'{stat}_Result'] = np.where(df[f'{stat}_line'].isnull(), np.nan, df[f'{stat}_Result'])
        df[f'{stat}_Diff'] = df[stat] - df[f'{stat}_line']
        cols.append(stat)
        cols.append(f'{stat}_line')
        cols.append(f'{stat}_Diff')
        cols.append(f'{stat}_Result')
    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    
    df_stats = pd.read_csv(f"../tables/{year}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    stats_cols = []
    
    for stat in stats:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    for stat in stats:
        stats_cols.extend([
            f"Off_{stat}",
            f"Off_L5_{stat}",
            f"AVG_{stat}_H2H",
            f"Def_{stat}",
            f"Def_L5_{stat}",
            f"df_stats.{stat}_line",
            f"{stat}",
            f"df.{stat}_Diff",
            f"{stat}_Result",
            f"{stat}_P_Diff",
            f"{stat}_Hit_Pct"
        ])
    df_select = ", ".join(stats_cols)
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, B2B, Spread, Total, df.Player, df.Pos, MP, Opp,
                                 {df_select} FROM df JOIN df_stats 
                                 ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                                 """).fetchdf()
    partition_save_df(df_actuals, f"../tables/{year}/parlay_actuals.csv")
    display(df_actuals)

    return df_actuals
                      
df = create_actuals_tbl(con)

# Parlay Performances

In [ ]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Spread, Total, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'O'
                            ORDER BY {stat}_Diff DESC
                            LIMIT 15""").fetchdf())

In [ ]:
for stat in ['PTS', 'AST', 'REB', 'PR', 'PA', 'RA', 'PRA']:
    print(f"==={stat}===")
    display(con.execute(f"""SELECT game_id, Date, Team, B2B, Spread, Total, Player, Pos, MP, Opp, 
                            Off_{stat}, Off_L5_{stat}, AVG_{stat}_H2H, 
                            Def_{stat}, Def_L5_{stat}, {stat}_line, {stat}, 
                            {stat}_Diff, {stat}_Result FROM df
                            WHERE {stat}_line IS NOT NULL AND {stat}_Result = 'U'
                            ORDER BY {stat}_Diff
                            LIMIT 15""").fetchdf())

# REPOPULATE DATA

In [4]:
def create_actuals_tbl2(con, now, season):
    
    df_log = pd.read_csv(f"../tables/{season}/season_gamelogs.csv")
    df_log['Date'] = pd.to_datetime(df_log.Date)
    df_log = df_log[(df_log.Date == now)]
    df_log = df_log.rename(columns={"TRB": "REB", "3PM": "TPM", "3PA": "TPA"})
    df_log['PR'] = df_log.PTS + df_log.REB 
    df_log['PA'] = df_log.PTS + df_log.AST
    df_log['RA'] = df_log.REB + df_log.AST
    df_log['PRA'] = df_log.PTS + df_log.REB + df_log.AST
    df_log['STL_BLK'] = df_log.STL + df_log.BLK
    df = df_log.copy()
    cols = []
    for stat in stats:
        cols.append(stat)

    df = df[['game_id', 'Date', 'Team', 'Player', 'Pos', 'MP'] + cols]
    df_stats = pd.read_csv(f"../tables/{season}/parlay_stats.csv")
    df_stats['Date'] = pd.to_datetime(df_stats.Date).astype(str)
    lines = []
    diffs = []
    stats_cols = []
    
    for stat in stats:
        lines.append(f'{stat}_line')
        diffs.append(f'{stat}_Diff')
    for stat in stats:
        stats_cols.extend([
            f"Off_{stat}",
            f"Off_L5_{stat}",
            f"AVG_{stat}_H2H",
            f"Def_{stat}",
            f"Def_L5_{stat}",
            f"{stat}",
        ])
    df_select = ", ".join(stats_cols)
    lines = str(lines).replace("[", "").replace("]", "")
    diffs = str(diffs).replace("[", "").replace("]", "")
    df_actuals = con.execute(f"""SELECT game_id, df.Date, df.Team, B2B, df.Player, df.Pos, MP, Opp,
                                 {df_select} FROM df JOIN df_stats 
                                 ON df.Date = df_stats.Date AND df.Player = df_stats.Player
                                 """).fetchdf()
    partition_save_df(df_actuals, f"../tables/{season}/parlay_actuals.csv")
#     display(df_actuals)

    return df_actuals

In [5]:
def repopulate_old_szns(season):
    schd = pd.read_csv(f"../tables/{season}/nba_schedule.csv")
    schd['Date'] = pd.to_datetime(schd.Date)
    # schd = schd[(schd.Date >= '2024-12-04')]
    df_lines = pd.read_csv(f"../tables/{season}/parlay_lines.csv")
    for date in schd.Date.unique().tolist():
        dt_str = date.strftime("%Y-%m-%d")
        now = dt_str
        print(now)
        create_actuals_tbl2(con, now, season)